In [546]:
import pandas as pd
import numpy as np
from scipy.stats import lognorm
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.metrics import mean_absolute_error

In [547]:
in_file_name = 'data/grouped_paces_ju.tsv'
df_all = pd.read_csv(in_file_name, delimiter="\t")
history = df_all

In [548]:
order18 = pd.read_csv('data/running_order_j2018_ju.tsv', delimiter="\t")

In [549]:
#paces = df.as_matrix(["pace_1", "pace_2", "pace_3", "pace_4", "pace_5", "pace_6"])
paces = history[["pace_1", "pace_2", "pace_3", "pace_4", "pace_5", "pace_6"]]
logs = np.log(paces)
means = np.nanmean(logs, axis=1)
stdevs = np.nanstd(logs, axis=1)
history = history.assign(log_means=pd.Series(means).values)
history = history.assign(log_stdevs=pd.Series(stdevs).values)

/Users/oskari/.virtualenvs/jukola-xml-model-PGRmw6ze/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: Mean of empty slice
  after removing the cwd from sys.path.
/Users/oskari/.virtualenvs/jukola-xml-model-PGRmw6ze/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1434: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


In [550]:
# Estimate values for all, but only use them if no history is available
with_history = history[np.isfinite(history.pace_2)]
x = with_history.mean_team_id.values
x = x.reshape(len(x), 1)

log_means = with_history.log_means.values.reshape(len(with_history.log_means), 1)
log_means_model = linear_model.LinearRegression()
log_means_model.fit(x, log_means)

estimated_log_means = log_means_model.predict(order18.team_id.values.reshape(len(order18.team_id), 1))
order18 = order18.assign(estimated_log_means=estimated_log_means)

log_stdevs = with_history.log_stdevs.values.reshape(len(with_history.log_stdevs), 1)
log_stdevs_model = linear_model.LinearRegression()
log_stdevs_model.fit(x, log_stdevs)

estimated_log_stdevs = log_stdevs_model.predict(order18.team_id.values.reshape(len(order18.team_id), 1))
order18 = order18.assign(estimated_log_stdevs=estimated_log_stdevs)
order18.head()

,team_id,team,team_base_name,leg,leg_dist,name,estimated_log_means,estimated_log_stdevs
0,5,Halden SK 1,Halden SK,1,11.0,Emil Wingstedt,1.944872,0.085008
1,5,Halden SK 1,Halden SK,2,11.9,Fredrik Eliasson,1.944872,0.085008
2,5,Halden SK 1,Halden SK,3,12.8,Thomas Natvig Årstad,1.944872,0.085008
3,5,Halden SK 1,Halden SK,4,8.7,Alexander Chepelin,1.944872,0.085008
4,5,Halden SK 1,Halden SK,5,8.7,Riccardo Scalet,1.944872,0.085008


In [551]:
# Combine history with 2018 runners 
no_history_row = pd.DataFrame([[0,0]], columns=["log_means", "log_stdevs"])
def get_history_row(running_order_row):
    name = running_order_row["name"].lower()
    
    by_name = history[history['name'] == name]
    by_name_and_colon = history[history['name'].str.contains(name + ":", regex=False)]

    runners = by_name.append(by_name_and_colon)
    if(len(runners) == 1):
        return runners
    team_name = running_order_row["team_base_name"].upper()
    runners = runners[runners['teams'].str.contains(team_name, regex=False)]
    if(len(runners) == 1):
        return runners
    if(len(runners) == 0):
        return no_history_row
    print(f"name '{name}' team_name '{team_name}'")
    print(f"by_name {len(by_name)} by_name_and_colon {len(by_name_and_colon)} runners {len(runners)}")
    print(f"Duplicate runner {runners}")
    #print(f"TEAMS by_name_and_colon {by_name_and_colon['teams']}")
    return runners.sort_values("num_runs", ascending = False).head(1)

def get_estimate_params(running_order_row):
    history_row = get_history_row(running_order_row)
    #print(f"estimate_row log_means {history_row.log_means} {history_row.log_stdevs}")
    log_means = history_row.log_means.values[0]
    log_stdevs = history_row.log_stdevs.values[0]
    return pd.Series({"history_log_means": log_means, "history_log_stdevs": log_stdevs})

#order18 = order18[order18['team'].str.contains("Reak") | order18['team'].str.contains("Puskasil") | order18['team'].str.contains("Rastihaukat")]
estimate_params = order18.apply(lambda row: get_estimate_params(row), axis=1)
order18 = order18.assign(history_log_means = estimate_params.history_log_means)
order18 = order18.assign(history_log_stdevs = estimate_params.history_log_stdevs)

In [552]:
order18['log_means'] = np.where(np.isfinite(order18["history_log_means"]) & order18["history_log_means"] > 0, order18["history_log_means"], order18["estimated_log_means"])
order18['log_stdevs'] = np.where(np.isfinite(order18["history_log_stdevs"]) & order18["history_log_stdevs"] > 0, order18["history_log_stdevs"], order18["estimated_log_stdevs"])
order18.head()


,team_id,team,team_base_name,leg,leg_dist,name,estimated_log_means,estimated_log_stdevs,history_log_means,history_log_stdevs,log_means,log_stdevs
0,5,Halden SK 1,Halden SK,1,11.0,Emil Wingstedt,1.944872,0.085008,1.763072,0.025436,1.763072,0.025436
1,5,Halden SK 1,Halden SK,2,11.9,Fredrik Eliasson,1.944872,0.085008,1.842846,0.093588,1.842846,0.093588
2,5,Halden SK 1,Halden SK,3,12.8,Thomas Natvig Årstad,1.944872,0.085008,1.797910,0.103600,1.797910,0.103600
3,5,Halden SK 1,Halden SK,4,8.7,Alexander Chepelin,1.944872,0.085008,1.929038,0.069687,1.929038,0.069687
4,5,Halden SK 1,Halden SK,5,8.7,Riccardo Scalet,1.944872,0.085008,1.958229,0.119586,1.958229,0.119586


In [553]:
# Calculate personal estimates
# s = sigma and scale = exp(mu).

log_means = np.exp(order18['log_means']) 
log_stdevs = order18['log_stdevs']

intervals95 = lognorm.interval(0.95, s = log_stdevs, scale = log_means)
means = lognorm.mean(s = log_stdevs, scale = log_means)
medians = lognorm.median(s = log_stdevs, scale = log_means)

In [554]:
order18 = order18.assign(ind_95_start = pd.Series(intervals95[0] * order18.leg_dist).values)
order18 = order18.assign(ind_95_end = pd.Series(intervals95[1] * order18.leg_dist).values)
order18 = order18.assign(ind_mean = pd.Series(means * order18.leg_dist).values)
order18 = order18.assign(ind_median = pd.Series(medians * order18.leg_dist).values)

In [555]:
by_teams = order18.set_index(["team_id", "leg"]).unstack()
by_teams.head()

team                              \
leg                               1                           2   
team_id                                                           
5                       Halden SK 1                 Halden SK 1   
11                       OK Linne 1                  OK Linne 1   
14       Ikaalisten Nouseva-Voima 1  Ikaalisten Nouseva-Voima 1   
17                     IL Tyrving 1                IL Tyrving 1   
18                    Nydalens SK 1               Nydalens SK 1   

                                                                 \
leg                               3                           4   
team_id                                                           
5                       Halden SK 1                 Halden SK 1   
11                       OK Linne 1                  OK Linne 1   
14       Ikaalisten Nouseva-Voima 1  Ikaalisten Nouseva-Voima 1   
17                     IL Tyrving 1                IL Tyrving 1   
18                    Nydalens SK 1               Nydalens SK 1   

                                                                 \
leg                               5                           6   
team_id                                                           
5                       Halden SK 1                 Halden SK 1   
11                       OK Linne 1                  OK Linne 1   
14       Ikaalisten Nouseva-Voima 1  Ikaalisten Nouseva-Voima 1   
17                     IL Tyrving 1                IL Tyrving 1   
18                    Nydalens SK 1               Nydalens SK 1   

                                               team_base_name  \
leg                               7                         1   
team_id                                                         
5                       Halden SK 1                 Halden SK   
11                       OK Linne 1                  OK Linne   
14       Ikaalisten Nouseva-Voima 1  Ikaalisten Nouseva-Voima   
17                     IL Tyrving 1                IL Tyrving   
18                    Nydalens SK 1               Nydalens SK   

                                                                ...      \
leg                             2                         3     ...       
team_id                                                         ...       
5                       Halden SK                 Halden SK     ...       
11                       OK Linne                  OK Linne     ...       
14       Ikaalisten Nouseva-Voima  Ikaalisten Nouseva-Voima     ...       
17                     IL Tyrving                IL Tyrving     ...       
18                    Nydalens SK               Nydalens SK     ...       

          ind_mean                        ind_median                        \
leg              5          6           7          1          2          3   
team_id                                                                      
5        62.097304  75.794950   85.633899  64.133501  75.142332  77.273814   
11       56.721972  67.114796   92.568771  68.047577  75.630890  74.017768   
14       62.576465  66.628415  102.489651  70.737369  77.901033  84.792953   
17       57.333536  67.761076   84.976476  65.716365  72.341834  72.698226   
18       50.992857  65.046605   94.260852  69.885691  77.440836  76.428838   

                                                      
leg              4          5          6           7  
team_id                                               
5        59.881118  61.654866  75.521582   85.578863  
11       51.993243  55.593390  66.933450   92.321787  
14       54.158064  62.080109  66.525176  101.902216  
17       53.732789  57.079184  67.619799   84.945242  
18       55.973405  50.808000  64.810800   93.949004  

[5 rows x 98 columns]

In [556]:
# remove teams missing some runners
print(len(by_teams))
by_teams = by_teams[np.isfinite(by_teams.log_means[1]) & np.isfinite(by_teams.log_means[2]) & np.isfinite(by_teams.log_means[3]) & np.isfinite(by_teams.log_means[4]) & np.isfinite(by_teams.log_means[5]) & np.isfinite(by_teams.log_means[6]) & np.isfinite(by_teams.log_means[7])] 
print(len(by_teams))

1103
1080


In [557]:
# Sum of log normal variables is not defined 
# so we simulate 10000 runs for each user and sum them and then do statistics on simulated results 
def simulate_relay_estimates(row):
    samples = pd.DataFrame()
    for i in range(1,8):
        if np.isnan(row["log_means"][i]):
            print(row["log_means"])
            print(row["name"])
        samples[i] = row["leg_dist"][i] * lognorm.rvs(s = row["log_stdevs"][i], scale = np.exp(row["log_means"][i]), size = 10000)

    samples_sums = pd.DataFrame()
    # leg_1 
    # leg_1 + leg_2
    # leg_1 + leg_2 + leg_3
    # ...
    for i in range(1,8):
        samples_sums[i] = np.sum([ samples[j] for j in range(1,i+1) ], axis=0)

    start95 = samples_sums.quantile(0.025)
    end95 = samples_sums.quantile(0.975)
    medians = samples_sums.median()
    means = samples_sums.mean()
    
    """
    for i in range(1,8):
        bins = int(samples_sums[i].max() - samples_sums[i].min())
        name = row["name"][i]
        plt.title(f"{name} bins = {bins}")
        plt.hist(samples_sums[i], bins=bins)
        #plt.axvline(x=row["fin_real"][i], color="r")        
        plt.axvline(x=medians[i], color="g")
        plt.axvline(x=means[i], color="yellow")
        plt.show()    
    """
    
    """
    bins = int( (samples_sums.max().max() - samples_sums.min().min()) / 5) 
    plt.figure(figsize=(20, 6))
    plt.title(f"Whole team, bins = {bins}")
    plt.hist([samples_sums[1], samples_sums[2], samples_sums[3], samples_sums[4], samples_sums[5], samples_sums[6], samples_sums[7]], bins=bins)
    for i in range(1,8):
        #plt.axvline(x=row["fin_real"][i], color="r")
        plt.axvline(x=medians[i], color="g")

    plt.show()
    """
    
    fin_start95_dict = {f"fin_start95_{leg}" : start95.values[leg-1] for leg in range(1,8)}
    fin_end95_dict = {f"fin_end95_{leg}" : end95.values[leg-1] for leg in range(1,8)}
    fin_median_dict = {f"fin_median_{leg}" : medians.values[leg-1] for leg in range(1,8)}
    fin_mean_dict = {f"fin_mean_{leg}" : means.values[leg-1] for leg in range(1,8)}
    new_cols = {**fin_start95_dict, **fin_end95_dict, **fin_median_dict, **fin_mean_dict}

    #print(start95.values)
    #print(new_cols)
    return pd.Series(new_cols)

relay_estimates = by_teams.apply(simulate_relay_estimates, axis=1)
relay_estimates.head()

,fin_start95_1,fin_start95_2,fin_start95_3,fin_start95_4,fin_start95_5,fin_start95_6,fin_start95_7,fin_end95_1,fin_end95_2,fin_end95_3,...,fin_median_5,fin_median_6,fin_median_7,fin_mean_1,fin_mean_2,fin_mean_3,fin_mean_4,fin_mean_5,fin_mean_6,fin_mean_7
team_id,,,,,,,,,,,,,,,,,,,,,
5,61.011822,126.119409,197.006493,255.172170,313.609179,386.184447,471.188304,67.356176,154.964169,239.928384,...,339.293772,415.055566,500.666726,64.157223,139.769020,217.382766,277.482081,339.749252,415.557045,501.203480
11,57.042944,125.959410,196.502413,247.947183,296.824836,362.202379,451.702764,80.708645,164.176394,242.302882,...,326.673918,393.813272,486.431096,68.257820,144.197291,218.506441,270.544109,327.278014,394.427454,486.944986
14,57.729771,131.238692,208.541093,261.864247,320.196883,386.027697,482.749231,86.458065,169.292041,263.831313,...,350.979539,417.408611,519.731953,71.057105,149.188756,234.588021,288.846766,351.352393,417.873189,520.337138
17,56.531588,127.506612,197.993500,248.332626,302.999599,368.972784,453.368693,76.278805,149.978670,225.543621,...,322.213805,389.852883,474.872089,65.856191,138.273901,211.121368,265.041828,322.396241,390.173812,475.178688
18,58.107386,127.855327,200.499738,255.059362,304.765878,367.748325,458.108031,83.831937,170.300240,250.723267,...,331.399464,396.471835,490.522862,70.244605,147.979347,224.553353,280.715594,331.753146,396.715564,491.027573


In [558]:
len(relay_estimates)

1080

In [559]:
# Flatten the troublesome multi-index to field_{leg} etc...
by_teams_flat = by_teams.copy()
by_teams_flat.columns = [f'{x[0]}_{x[1]}' for x in by_teams_flat.columns]
by_teams_flat.reset_index()


,team_id,team_1,team_2,team_3,team_4,team_5,team_6,team_7,team_base_name_1,team_base_name_2,...,ind_mean_5,ind_mean_6,ind_mean_7,ind_median_1,ind_median_2,ind_median_3,ind_median_4,ind_median_5,ind_median_6,ind_median_7
0,5,Halden SK 1,Halden SK 1,Halden SK 1,Halden SK 1,Halden SK 1,Halden SK 1,Halden SK 1,Halden SK,Halden SK,...,62.097304,75.794950,85.633899,64.133501,75.142332,77.273814,59.881118,61.654866,75.521582,85.578863
1,11,OK Linne 1,OK Linne 1,OK Linne 1,OK Linne 1,OK Linne 1,OK Linne 1,OK Linne 1,OK Linne,OK Linne,...,56.721972,67.114796,92.568771,68.047577,75.630890,74.017768,51.993243,55.593390,66.933450,92.321787
2,14,Ikaalisten Nouseva-Voima 1,Ikaalisten Nouseva-Voima 1,Ikaalisten Nouseva-Voima 1,Ikaalisten Nouseva-Voima 1,Ikaalisten Nouseva-Voima 1,Ikaalisten Nouseva-Voima 1,Ikaalisten Nouseva-Voima 1,Ikaalisten Nouseva-Voima,Ikaalisten Nouseva-Voima,...,62.576465,66.628415,102.489651,70.737369,77.901033,84.792953,54.158064,62.080109,66.525176,101.902216
3,17,IL Tyrving 1,IL Tyrving 1,IL Tyrving 1,IL Tyrving 1,IL Tyrving 1,IL Tyrving 1,IL Tyrving 1,IL Tyrving,IL Tyrving,...,57.333536,67.761076,84.976476,65.716365,72.341834,72.698226,53.732789,57.079184,67.619799,84.945242
4,18,Nydalens SK 1,Nydalens SK 1,Nydalens SK 1,Nydalens SK 1,Nydalens SK 1,Nydalens SK 1,Nydalens SK 1,Nydalens SK,Nydalens SK,...,50.992857,65.046605,94.260852,69.885691,77.440836,76.428838,55.973405,50.808000,64.810800,93.949004
5,26,NTNUI 1,NTNUI 1,NTNUI 1,NTNUI 1,NTNUI 1,NTNUI 1,NTNUI 1,NTNUI,NTNUI,...,61.617233,72.004349,101.795724,68.011089,76.987816,80.864879,66.259730,61.393158,71.633572,101.693527
6,28,Angelniemen Ankkuri 1,Angelniemen Ankkuri 1,Angelniemen Ankkuri 1,Angelniemen Ankkuri 1,Angelniemen Ankkuri 1,Angelniemen Ankkuri 1,Angelniemen Ankkuri 1,Angelniemen Ankkuri,Angelniemen Ankkuri,...,58.784104,67.740436,108.455636,85.074000,81.385672,79.706644,55.722948,58.474682,67.436225,108.060920
7,32,OK Orion 1,OK Orion 1,OK Orion 1,OK Orion 1,OK Orion 1,OK Orion 1,OK Orion 1,OK Orion,OK Orion,...,58.861257,76.690217,97.081819,78.620226,74.145082,81.015163,58.979693,58.646700,76.410671,96.672657
8,35,OLV Baselland 1,OLV Baselland 1,OLV Baselland 1,OLV Baselland 1,OLV Baselland 1,OLV Baselland 1,OLV Baselland 1,OLV Baselland,OLV Baselland,...,57.324702,61.586819,86.896183,62.876000,76.362300,83.038686,53.400600,57.115500,61.538667,86.667308
9,37,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,Järla Orientering,Järla Orientering,...,58.862402,78.894655,95.667554,69.616274,69.794621,84.531260,57.710983,58.602492,78.501248,95.300935


In [560]:
estimates = pd.concat([by_teams_flat, relay_estimates], axis=1, join='inner')

In [561]:
# Convert minutes to date and times
start_timestamp = pd.Timestamp(year = 2018, month = 6, day = 16, hour = 23)

for leg in range(1,8):
    estimates[f"fint_median_{leg}"] = pd.to_datetime(estimates[f"fin_median_{leg}"] * 60, unit = "s", origin= start_timestamp)
    estimates[f"fint_start95_{leg}"] = pd.to_datetime(estimates[f"fin_start95_{leg}"] * 60, unit = "s", origin= start_timestamp)
    estimates[f"fint_end95_{leg}"] = pd.to_datetime(estimates[f"fin_end95_{leg}"] * 60, unit = "s", origin= start_timestamp)


In [562]:
# Sort teams by estimated total time 
estimates = estimates.sort_values("fin_median_7")

estimates.to_csv('data/team_estimates_ju2018.tsv', sep="\t")

In [563]:
estimates[["team_1", "fin_median_7"]].head(10)

,team_1,fin_median_7
team_id,,
17,IL Tyrving 1,474.872089
35,OLV Baselland 1,482.086726
11,OK Linne 1,486.431096
18,Nydalens SK 1,490.522862
5,Halden SK 1,500.666726
37,Järla Orientering 1,515.841660
14,Ikaalisten Nouseva-Voima 1,519.731953
150,IL Tyrving 2,525.712638
32,OK Orion 1,526.881121


In [564]:
teams_to_follow = estimates[estimates['team_1'].str.contains("Reak") | estimates['team_1'].str.contains("Puskasil") | estimates['team_1'].str.contains("Rastihaukat")]
teams_to_follow[["team_1", "fin_median_7", "fin_start95_7", "fin_end95_7"]]

,team_1,fin_median_7,fin_start95_7,fin_end95_7
team_id,,,,
429,Reaktor Innovations 1,744.737456,697.834082,795.780570
568,Rastihaukat 1,752.535257,698.969535,809.234264
1270,Reaktor Innovations 2,883.100123,795.039929,991.569795
1246,Rastihaukat 2,910.284999,832.877323,993.044510
1131,Puskasilimät OK 1,913.282587,856.268923,976.293168


In [565]:
leg_1_cols = list(filter(lambda c: "_1" in c,estimates.columns.values))
column_base_names = list(map(lambda c: c[:-2], leg_1_cols))
runner_estimates = pd.wide_to_long(estimates.reset_index(), stubnames=column_base_names, i ="team_id", j="leg", sep = "_").sort_values(by=['team_id', 'leg'])
runner_estimates = runner_estimates.drop(['team_base_name', 'estimated_log_means', 'estimated_log_stdevs'], axis=1)
runner_estimates.head()

team  leg_dist                  name  history_log_means  \
team_id leg                                                                   
5       1    Halden SK 1      11.0        Emil Wingstedt           1.763072   
        2    Halden SK 1      11.9      Fredrik Eliasson           1.842846   
        3    Halden SK 1      12.8  Thomas Natvig Årstad           1.797910   
        4    Halden SK 1       8.7    Alexander Chepelin           1.929038   
        5    Halden SK 1       8.7       Riccardo Scalet           1.958229   

             history_log_stdevs  log_means  log_stdevs  ind_95_start  \
team_id leg                                                            
5       1              0.025436   1.763072    0.025436     61.014607   
        2              0.093588   1.842846    0.093588     62.549244   
        3              0.103600   1.797910    0.103600     63.073600   
        4              0.069687   1.929038    0.069687     52.236299   
        5              0.119586   1.958229    0.119586     48.772555   

             ind_95_end   ind_mean  ind_median  fin_start95   fin_end95  \
team_id leg                                                               
5       1     67.411824  64.154251   64.133501    61.011822   67.356176   
        2     90.270797  75.472132   75.142332   126.119409  154.964169   
        3     94.671024  77.689619   77.273814   197.006493  239.928384   
        4     68.644760  60.026693   59.881118   255.172170  301.740330   
        5     77.939788  62.097304   61.654866   313.609179  368.221232   

             fin_median    fin_mean                fint_median  \
team_id leg                                                      
5       1     64.146599   64.157223 2018-06-17 00:04:08.795930   
        2    139.466230  139.769020 2018-06-17 01:19:27.973798   
        3    216.997295  217.382766 2018-06-17 02:36:59.837712   
        4    277.055437  277.482081 2018-06-17 03:37:03.326220   
        5    339.293772  339.749252 2018-06-17 04:39:17.626322   

                          fint_start95                 fint_end95  
team_id leg                                                        
5       1   2018-06-17 00:01:00.709339 2018-06-17 00:07:21.370585  
        2   2018-06-17 01:06:07.164533 2018-06-17 01:34:57.850113  
        3   2018-06-17 02:17:00.389569 2018-06-17 02:59:55.703052  
        4   2018-06-17 03:15:10.330175 2018-06-17 04:01:44.419813  
        5   2018-06-17 04:13:36.550759 2018-06-17 05:08:13.273930

In [566]:
runner_estimates.to_csv('data/runner_estimates_ju2018.tsv', sep="\t")

In [567]:
for_print = runner_estimates.copy()
for_print = for_print.reset_index()
for_print = for_print.set_index('team_id')
for_print = for_print.round(1)
for_print.fint_median = for_print.fint_median.dt.strftime("%H:%M")
for_print.fint_start95 = for_print.fint_start95.dt.strftime("%H:%M")
for_print.fint_end95 = for_print.fint_end95.dt.strftime("%H:%M")
for_print = for_print[['team',
 'leg',
 'name',
 'ind_median', 
 'ind_95_start', 
 'ind_95_end', 
 'fin_median',
 'fint_median',
 'fint_start95',
 'fint_end95']]
for_print.head()

,team,leg,name,ind_median,ind_95_start,ind_95_end,fin_median,fint_median,fint_start95,fint_end95
team_id,,,,,,,,,,
5,Halden SK 1,1,Emil Wingstedt,64.1,61.0,67.4,64.1,00:04,00:01,00:07
5,Halden SK 1,2,Fredrik Eliasson,75.1,62.5,90.3,139.5,01:19,01:06,01:34
5,Halden SK 1,3,Thomas Natvig Årstad,77.3,63.1,94.7,217.0,02:36,02:17,02:59
5,Halden SK 1,4,Alexander Chepelin,59.9,52.2,68.6,277.1,03:37,03:15,04:01
5,Halden SK 1,5,Riccardo Scalet,61.7,48.8,77.9,339.3,04:39,04:13,05:08


In [568]:
for_print.to_csv('for_print_ju2018.tsv', sep="\t")

In [569]:
for_print[for_print['team'].str.contains("Reak") | for_print['team'].str.contains("Puskasil") | for_print['team'].str.contains("Rastihaukat 2")]


,team,leg,name,ind_median,ind_95_start,ind_95_end,fin_median,fint_median,fint_start95,fint_end95
team_id,,,,,,,,,,
429,Reaktor Innovations 1,1,Mikko Peltonen,97.8,83.4,114.7,97.9,00:37,00:23,00:54
429,Reaktor Innovations 1,2,Pasi Huhtiniemi,109.8,95.3,126.6,207.9,02:27,02:06,02:50
429,Reaktor Innovations 1,3,Joni Freeman,107.8,86.2,134.8,316.2,04:16,03:45,04:50
429,Reaktor Innovations 1,4,Tuomas Kareinen,86.2,63.0,118.0,403.3,05:43,05:02,06:28
429,Reaktor Innovations 1,5,Janne Vaittinen,88.9,82.2,96.1,492.3,07:12,06:31,07:58
429,Reaktor Innovations 1,6,Olavi Kanerva,90.8,75.8,108.7,583.4,08:43,07:59,09:33
429,Reaktor Innovations 1,7,Jyri Kytömäki,161.3,146.7,177.4,744.7,11:24,10:37,12:15
1131,Puskasilimät OK 1,1,Tuomas Ketonen,124.3,101.4,152.4,124.3,01:04,00:41,01:32
1131,Puskasilimät OK 1,2,Petri Miettinen,131.1,122.1,140.6,255.5,03:15,02:50,03:45
